In [1]:
### Loading the Dataset

try: from datasets import load_dataset
except:
    !pip install datasets
    from datasets import load_dataset


dataset = load_dataset("nvidia/OpenMathInstruct-2", split = "train_1M")
dataset_small = dataset.select(range(10000))
dataset_split = dataset_small.train_test_split(test_size=0.1, seed=42)
train_data, eval_data = dataset_split["train"], dataset_split["test"]

print(dataset)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.41k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

...

Generating train split:   0%|          | 0/13972791 [00:00<?, ? examples/s]

Generating train_1M split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating train_2M split:   0%|          | 0/2000000 [00:00<?, ? examples/s]

Generating train_5M split:   0%|          | 0/5000000 [00:00<?, ? examples/s]

Dataset({
    features: ['problem', 'generated_solution', 'expected_answer', 'problem_source'],
    num_rows: 1000000
})


In [2]:
dataset_small.to_pandas().head()

,problem,generated_solution,expected_answer,problem_source
0,Solve for $y$:\n\n$$\frac{y^2 - 3y + 2}{y - 2}...,Start by multiplying both sides by $y - 2$ to ...,2,augmented_math
1,"Given a circle centered at the origin, a point...","The rotation matrix for rotating a point $(x, ...","(-\frac{\sqrt{2}}{2}, \frac{7\sqrt{2}}{2})",augmented_math
2,The repeating decimal $0.\overline{009}$ can b...,"To express $0.\overline{009}$ as a fraction, l...",224,augmented_math
3,Let $m$ and $n$ be two positive integers such ...,## Step 1: Recall the relationship between GCD...,4,augmented_math
4,"Let $(x, y)$ be a solution to the system of eq...",We have two equations:\n\[ x^2 + \lfloor y \rf...,9,augmented_math


In [3]:
### Logging into HuggingFace

try:
  from dotenv import load_dotenv
except:
  !pip install python-dotenv
  from dotenv import load_dotenv

from huggingface_hub import login, whoami
import os

load_dotenv()
login(os.getenv("hugging_face_key2"))

In [4]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, IA3Config, TaskType


model_name = "gpt2"
# model_name = "tiiuae/falcon-7b"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

peft_config = IA3Config(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["c_attn", "mlp.c_fc"],  
    feedforward_modules=["mlp.c_fc"],       
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 14,772,480 || all params: 139,212,288 || trainable%: 10.6115


In [6]:
def format_prompt(example):
    text = f"[INST] Problem: {example['problem']} [/INST] Solution: {example['generated_solution']}"
    tokenized = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train = train_data.map(format_prompt, batched=False, remove_columns=train_data.column_names)
tokenized_eval = eval_data.map(format_prompt, batched=False, remove_columns=eval_data.column_names)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling



training_args = TrainingArguments(
    output_dir=f"./IA3_{model_name}",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=1000,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",

)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval
)


trainer.train()



No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
100,1.191600,1.104002
200,1.194400,1.095721
300,1.176900,1.086805
400,1.174400,1.081791
500,1.136100,1.076548
600,1.168400,1.071763
700,1.142000,1.066210
800,1.109600,1.061868
900,1.152100,1.059689
1000,1.140000,1.055091


TrainOutput(global_step=3375, training_loss=1.122703592936198, metrics={'train_runtime': 919.578, 'train_samples_per_second': 29.361, 'train_steps_per_second': 3.67, 'total_flos': 7054884864000000.0, 'train_loss': 1.122703592936198, 'epoch': 3.0})

In [ ]:
peft_model.save_pretrained(f"./IA3_{model_name}_final")
tokenizer.save_pretrained(f"./IA3_{model_name}_final")

('./prefix_gpt2_final/tokenizer_config.json',
 './prefix_gpt2_final/special_tokens_map.json',
 './prefix_gpt2_final/vocab.json',
 './prefix_gpt2_final/merges.txt',
 './prefix_gpt2_final/added_tokens.json',
 './prefix_gpt2_final/tokenizer.json')

In [ ]:
from peft import PeftModel, PeftConfig
peft_config = PeftConfig.from_pretrained(f"./IA3_{model_name}_final")

base_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, f"./IA3_{model_name}_final")

tokenizer = AutoTokenizer.from_pretrained(f"./IA3_{model_name}_final")

In [12]:
model.eval()
import torch
from IPython.display import display, Math
prompt = "[INST] Solve for $y$: $$\frac{y^2 - 3y + 2}{y - 2} = y + 1$$ [/INST] Solution:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.95
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:1889: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


[INST] Solve for $y$: $$rac{y^2 - 3y + 2}{y - 2} = y + 1$$ [/INST] Solution: To solve for $y$, we can use the formula:
\[ y^2 - 3y + 2}{y - 2} = y + 1 \]

Since $y = 1$, we can write:
\[ y = \frac{y}{3} \]

So, $y = \boxed{3}$.<|endoftext|>


In [ ]:
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
Math(answer)